# E-Commerce with Virtual Mouse

In [1]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np
import math
import time
import webbrowser
import threading
from flask import Flask, request

Configuration

In [2]:
WEBCAM_ID = 0
CAM_WIDTH = 640
CAM_HEIGHT = 480
SMOOTHING = 7
FRAME_REDUCTION = 100
SERVER_PORT = 5000

Global Shutdown Event for Threading

In [3]:
shutdown_event = threading.Event()

HTML for the E-Commerce Frontend

In [4]:
HTML_CONTENT = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GestureMart - Python Edition</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">
    <style>
        body { font-family: 'Inter', sans-serif; }
        .product-card:hover {
            transform: translateY(-5px);
            box-shadow: 0 10px 15px -3px rgb(0 0 0 / 0.1), 0 4px 6px -4px rgb(0 0 0 / 0.1);
        }
        .product-card { transition: transform 0.3s ease, box-shadow 0.3s ease; }
    </style>
</head>
<body class="bg-gray-100 text-gray-800">
    <div class="container mx-auto px-4 py-8">
        <header class="flex justify-between items-center mb-8 pb-4 border-b border-gray-300">
            <h1 class="text-4xl font-bold text-indigo-600">GestureMart</h1>
            <div class="text-lg">
                <span class="font-semibold">Cart:</span>
                <span id="cart-count">0</span> items
            </div>
        </header>
        <div id="products" class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-8">
            <div class="bg-white rounded-lg shadow-md p-6 product-card">
                <img src="https://placehold.co/400x300/e2e8f0/4a5568?text=Smart+Watch" alt="Smart Watch" class="rounded-md mb-4 w-full h-48 object-cover">
                <h2 class="text-2xl font-bold mb-2">Futura Watch</h2><p class="text-gray-600 mb-4">The smartwatch that knows you better.</p>
                <div class="flex justify-between items-center"><span class="text-3xl font-bold text-indigo-600">$299</span><button class="bg-indigo-500 text-white font-bold py-2 px-4 rounded-lg hover:bg-indigo-600 transition-colors">Add to Cart</button></div>
            </div>
            <div class="bg-white rounded-lg shadow-md p-6 product-card">
                <img src="https://placehold.co/400x300/dbeafe/1e3a8a?text=VR+Headset" alt="VR Headset" class="rounded-md mb-4 w-full h-48 object-cover">
                <h2 class="text-2xl font-bold mb-2">Portal VR</h2><p class="text-gray-600 mb-4">Escape reality. Or just make it better.</p>
                <div class="flex justify-between items-center"><span class="text-3xl font-bold text-indigo-600">$450</span><button class="bg-indigo-500 text-white font-bold py-2 px-4 rounded-lg hover:bg-indigo-600 transition-colors">Add to Cart</button></div>
            </div>
            <div class="bg-white rounded-lg shadow-md p-6 product-card">
                <img src="https://placehold.co/400x300/e0f2fe/0891b2?text=AI+Speaker" alt="AI Speaker" class="rounded-md mb-4 w-full h-48 object-cover">
                <h2 class="text-2xl font-bold mb-2">EchoSphere</h2><p class="text-gray-600 mb-4">It's always listening. In a helpful way.</p>
                <div class="flex justify-between items-center"><span class="text-3xl font-bold text-indigo-600">$129</span><button class="bg-indigo-500 text-white font-bold py-2 px-4 rounded-lg hover:bg-indigo-600 transition-colors">Add to Cart</button></div>
            </div>
        </div>
        <div class="text-center mt-12"><button id="buy-now" class="bg-emerald-500 text-white font-bold py-4 px-8 rounded-lg text-2xl hover:bg-emerald-600 transition-colors shadow-lg">Buy Now</button></div>
    </div>
    <div id="success-modal" class="hidden fixed inset-0 bg-gray-900 bg-opacity-75 flex items-center justify-center z-50">
        <div class="bg-white p-8 rounded-lg shadow-2xl text-center">
            <h2 class="text-3xl font-bold text-emerald-500 mb-4">Thank You!</h2>
            <p class="text-lg text-gray-700">Items Bought Successfully!</p>
            <p class="text-sm text-gray-500 mt-2">Application will close in <span id="countdown">10</span> seconds.</p>
        </div>
    </div>
    <script>
        let cartCount = 0;
        const cartCountEl = document.getElementById('cart-count');
        document.getElementById('products').addEventListener('click', (e) => {
            if (e.target.tagName === 'BUTTON' && e.target.textContent === 'Add to Cart') {
                cartCount++;
                cartCountEl.textContent = cartCount;
            }
        });
        document.getElementById('buy-now').addEventListener('click', () => {
            if (cartCount > 0) {
                document.getElementById('success-modal').classList.remove('hidden');
                let seconds = 10;
                const countdownEl = document.getElementById('countdown');
                countdownEl.textContent = seconds;
                const interval = setInterval(() => {
                    seconds--;
                    countdownEl.textContent = seconds;
                    if (seconds <= 0) {
                        clearInterval(interval);
                        // This is the key change: send a request to our Python server to shut down.
                        fetch('/shutdown').then(() => {
                            document.body.innerHTML = '<div class="w-screen h-screen flex items-center justify-center bg-gray-900 text-white text-2xl">Session Ended. Closing...</div>';
                        });
                    }
                }, 1000);
            } else {
                alert("Your cart is empty!");
            }
        });
    </script>
</body>
</html>
"""

Flask Web Server

In [5]:
app = Flask(__name__)


@app.route('/')
# Home Page
def home():
    return HTML_CONTENT


@app.route('/shutdown', methods=['GET'])
# Endpoint to trigger application shutdown
def shutdown():
    print("Shutdown signal received from browser.")
    shutdown_event.set()
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()
    return 'Server shutting down...'


# Runs the Flask app in a separate thread
def run_flask_app():
    app.run(port=SERVER_PORT, debug=False, use_reloader=False) # Disable reloader to prevent the script from running twice

AI Virtual Mouse Functionality

In [6]:
# Main function
def run_virtual_mouse():
    p_time = 0
    p_loc_x, p_loc_y = 0, 0
    c_loc_x, c_loc_y = 0, 0
    last_scroll_time = 0

    cap = cv2.VideoCapture(WEBCAM_ID)
    if not cap.isOpened():
        print(f"Error: Could not open webcam with ID {WEBCAM_ID}.")
        return
    cap.set(3, CAM_WIDTH)
    cap.set(4, CAM_HEIGHT)

    screen_width, screen_height = pyautogui.size()
    pyautogui.FAILSAFE = False

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.5)
    mp_draw = mp.solutions.drawing_utils

    print("AI Virtual Mouse thread started.")

    while not shutdown_event.is_set():
        success, frame = cap.read()
        if not success:
            continue

        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]

            # Get landmark coordinates
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
            index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
            middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
            ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]

            h, w, c = frame.shape
            ix, iy = int(index_tip.x * w), int(index_tip.y * h)

            # Cursor Movement
            screen_x = np.interp(ix, (FRAME_REDUCTION, w - FRAME_REDUCTION), (0, screen_width))
            screen_y = np.interp(iy, (FRAME_REDUCTION, h - FRAME_REDUCTION), (0, screen_height))
            c_loc_x = p_loc_x + (screen_x - p_loc_x) / SMOOTHING
            c_loc_y = p_loc_y + (screen_y - p_loc_y) / SMOOTHING
            pyautogui.moveTo(c_loc_x, c_loc_y)
            p_loc_x, p_loc_y = c_loc_x, c_loc_y

            # Click Gesture
            click_distance = math.hypot(middle_tip.x - index_tip.x, middle_tip.y - index_tip.y)
            if click_distance < 0.05:
                pyautogui.click()
                cv2.circle(frame, (ix, iy), 15, (0, 255, 0), cv2.FILLED)
                time.sleep(0.2) # Debounce click

            # Scroll Gesture (3 fingers up)
            if (index_tip.y < index_mcp.y and
                middle_tip.y < middle_mcp.y and
                ring_tip.y < ring_mcp.y):

                # Scroll up if hand moves up, scroll down if hand moves down
                scroll_amount = int((index_mcp.y - index_tip.y) * 20) # Negative to invert scroll direction
                pyautogui.scroll(scroll_amount)
                cv2.putText(frame, "SCROLLING", (150, 50), cv2.FONT_HERSHEY_PLAIN, 3, (0, 165, 255), 3)
                # Add a small delay to prevent excessive scrolling
                time.sleep(0.1)

            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display FPS
        c_time = time.time()
        fps = 1 / (c_time - p_time) if (c_time - p_time) > 0 else 0
        p_time = c_time
        cv2.putText(frame, f'FPS: {int(fps)}', (20, 50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)

        cv2.imshow("AI Virtual Mouse - Press 'q' in this window to force quit", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            shutdown_event.set() # Also allow manual quit
            break

    # Cleanup
    print("Virtual mouse thread shutting down...")
    cap.release()
    cv2.destroyAllWindows()
    hands.close()

Main Execution Block

In [7]:
if __name__ == '__main__':
    print("Starting application...")

    # Start the Flask server in a daemon thread
    flask_thread = threading.Thread(target=run_flask_app, daemon=True)
    flask_thread.start()
    print(f"Web server started on http://127.0.0.1:{SERVER_PORT}")

    # Start the virtual mouse in another thread
    mouse_thread = threading.Thread(target=run_virtual_mouse)
    mouse_thread.start()

    # Open the web browser to the e-commerce page
    time.sleep(1) # Give the server a moment to start
    webbrowser.open_new(f"http://127.0.0.1:{SERVER_PORT}")
    print("Browser opened. The application is now running.")
    print("Control the mouse with your hand. Close the browser or use the 'Buy Now' flow to exit.")

    # Wait for the shutdown signal
    shutdown_event.wait()
    print("Main thread received shutdown signal. Terminating.")

    # Wait for the mouse thread to finish its cleanup
    mouse_thread.join()

    print("Application has terminated successfully.")

Starting application...
Web server started on http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Browser opened. The application is now running.
Control the mouse with your hand. Close the browser or use the 'Buy Now' flow to exit.
AI Virtual Mouse thread started.


127.0.0.1 - - [21/Jul/2025 22:57:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2025 22:57:59] "GET /favicon.ico HTTP/1.1" 404 -
[2025-07-21 22:59:19,901] ERROR in app: Exception on /shutdown [GET]
Traceback (most recent call last):
  File "C:\Projects\Virtual_Mouse\.venv\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Projects\Virtual_Mouse\.venv\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Projects\Virtual_Mouse\.venv\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Projects\Virtual_Mouse\.venv\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "C:\Users\sagni\AppData\Local\Temp\ipykernel_28760\2679668277.py", line 17, in shutdown
    raise RuntimeError('Not run

Shutdown signal received from browser.
Main thread received shutdown signal. Terminating.
Virtual mouse thread shutting down...
Application has terminated successfully.
